In [1]:
import numpy as np
from os.path import normpath as fn # Fix Linux/Windows path issue
import sys
sys.path.append("nn") # add the nn module into system path

import nn.layer as layer
import nn.graph as graph
import nn.solver as solver
import nn.container as container
import nn.loss as loss

np.random.seed(0)

In [2]:
# Load data
data = np.load(fn('data/mnist_26k.npz'))
train_im = np.float32(data['im_train'])/255.-0.5
train_im = np.reshape(train_im,[-1,28,28,1])
train_lb = data['lbl_train']

val_im = np.float32(data['im_val'])/255.-0.5
val_im = np.reshape(val_im,[-1,28,28,1])
val_lb = data['lbl_val']

In [3]:
###################################
# build static computational graph
###################################
C1 = 8
C2 = 16

# placeholder for input
inp = layer.Tensor()
lab = layer.Tensor()

model = container.Sequential([layer.Conv2d(1, C1, kernel_size=3, stride=1, padding=1, bias=True),
                           layer.RELU(),
                           layer.Maxpool2d(),
                           layer.Conv2d(C1, C2, kernel_size=3, stride=1, padding=1, bias=True),
                           layer.RELU(),
                           layer.Maxpool2d(),
                           layer.Dropout(0.3),
                           layer.Flatten(),
                           layer.Linear(C2*49 , 10)
                          ])
print(model)

# loss function: softmax + crossentropy
smaxloss = loss.SmaxCELoss()
accuracy = loss.Accuracy()

# build computational graph
y = model(inp)

# Cross Entropy of Soft-max
criterion = smaxloss(y,lab)

# Accuracy
acc = accuracy(y,lab)

Sequential(
  (0) Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1, bias=True)
  (1) RELU()
  (2) Maxpool2d(kernel_size=2, stride=2, padding=0)
  (3) Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1, bias=True)
  (4) RELU()
  (5) Maxpool2d(kernel_size=2, stride=2, padding=0)
  (6) Dropout(p=0.3)
  (7) Flatten()
  (8) Linear(in_features=784, out_features=10)
)


In [4]:
lr=0.01
NUM_EPOCH=2
DISPITER=100
BSZ=64

optimizer = solver.Momentum(graph.params, lr, mom=0.9)

In [5]:
# Training loop
batches = range(0,len(train_lb)-BSZ+1,BSZ)
niter, avg_loss, avg_acc=0, 0., 0.

for ep in range(NUM_EPOCH+1):
    graph.eval()
    # As we train, let's keep track of val accuracy
    vacc, vloss, viter= 0., 0., 0
    for b in range(0,len(val_lb)-BSZ+1,BSZ):
        inp.set(val_im[b:b+BSZ,...])
        lab.set(val_lb[b:b+BSZ])
        graph.Forward()
        viter += 1
        vacc += acc.top
        vloss += criterion.top
    vloss, vacc = vloss / viter, vacc / viter * 100
    print("%09d: #### %d Epochs: Val Loss = %.3e, Accuracy = %.2f%%" % (niter,ep,vloss,vacc))
    if ep == NUM_EPOCH:
        break
    
    graph.train()
    # Shuffle Training Set
    idx = np.random.permutation(len(train_lb))

    # Train one epoch
    for b in batches:
        # Load a batch
        inp.set(train_im[idx[b:b+BSZ],...])
        lab.set(train_lb[idx[b:b+BSZ]])

        graph.Forward()
        avg_loss += criterion.top 
        avg_acc += acc.top
        niter += 1
        if niter % DISPITER == 0:
            avg_loss = avg_loss / DISPITER
            avg_acc = avg_acc / DISPITER * 100
            print("%09d: Training Loss = %.3e, Accuracy = %.2f%%" % (niter,avg_loss,avg_acc))
            avg_loss, avg_acc = 0., 0.

        graph.Backward(criterion)
        optimizer.step()
        

AttributeError: 'Dropout' object has no attribute 'training'